In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [3]:
!pip install -q faiss-cpu

In [4]:
!pip install -q tiktoken

In [5]:
from langchain.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI

In [6]:
# load the data
loader=TextLoader("illuminati.txt")
document=loader.load()

In [7]:
#chunking the data
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
chunks=text_splitter.split_documents(document)

In [8]:
chunks

[Document(metadata={'source': 'illuminati.txt'}, page_content='The Illuminati (/əˌluːmɪˈnɑːti/; plural of Latin illuminatus, \'enlightened\') is a name given to several groups, both real and fictitious. Historically, the name usually refers to the Bavarian Illuminati, an Enlightenment-era secret society founded on 1 May 1776 in the Electorate of Bavaria. The society\'s stated goals were to oppose superstition, obscurantism, religious influence over public life, and abuses of state power by monarchs. "The order of the day", they wrote in their general statutes,'),
 Document(metadata={'source': 'illuminati.txt'}, page_content='the day", they wrote in their general statutes, "is to put an end to the machinations of the purveyors of injustice, to control them without dominating them."[2]'),
 Document(metadata={'source': 'illuminati.txt'}, page_content='The Illuminati—along with Freemasonry and other secret societies—were outlawed through edict by Charles Theodore, Elector of Bavaria, with 

In [2]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

/home/shahul/miniconda3/envs/LCenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
from langchain_community.vectorstores import FAISS
vectordb=FAISS.from_documents(chunks, embeddings)

In [11]:
retriever=vectordb.as_retriever()

In [12]:
llm=ChatOpenAI(temperature=0.6)

In [ ]:
template = """You are an assistant for question-answering tasks
Use the following pieces of retrieved context to answer the question.
If you dont know the answer, just say you dont know
use two sentences maximum and keep the answer concise
Question:{question}
context:{context}
Answer:
"""

In [14]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [15]:
prompt=ChatPromptTemplate.from_template(template)

In [16]:
#setting up RAG Pipeline

chain=(
    {"context": retriever, "question":RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser() 
)


In [17]:
chain.invoke("what were these organizations accused of?")

'These organizations were accused of conspiring to control world affairs, gain political power, influence, and establish a New World Order. They were also accused of masterminding events and planting agents in governments and corporations.'

In [18]:
questions=[
    "what were these organizations accused of?",
    "who was the non-clerical professor at the institution",
    "Who founded the Bavarian Illuminati and when?",
    "What symbol did the Perfectibilists (early Illuminati) adopt?"
]

In [19]:
ground_truth=[
    "The organizations were accused of conspiring to control world affairs and establish a New World Order. They were also accused of masterminding events and planting agents in governments and corporations to gain political power and influence.",
    "Adam Weishaupt (1748–1830) became professor of Canon Law and practical philosophy at the University of Ingolstadt in 1773.",
    "Adam Weishaupt founded the Bavarian Illuminati on 1 May 1776 in the Electorate of Bavaria.",
    "The Perfectibilists adopted the Owl of Minerva as their symbol."
]

In [20]:
answer=[]
content=[]

In [21]:
#inference

for query in questions:
    answer.append(chain.invoke(query))
    content.append([docs.page_content for docs in retriever.get_relevant_documents(query)])

/tmp/ipykernel_8766/2829473142.py:5: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  content.append([docs.page_content for docs in retriever.get_relevant_documents(query)])


In [22]:
content

[['In subsequent use, "Illuminati" has been used when referring to various organisations alleged to be a continuation of the original Bavarian Illuminati (though these links have not been substantiated). These organisations have often been accused of conspiring to control world affairs, by masterminding events and planting agents in governments and corporations, in order to gain political power, influence and to establish a New World Order. Central to some of the more widely known and elaborate',
  'The Illuminati—along with Freemasonry and other secret societies—were outlawed through edict by Charles Theodore, Elector of Bavaria, with the encouragement of the Catholic Church, in 1784, 1785, 1787 and 1790.[3] During subsequent years, the group was generally vilified by conservative and religious critics, who claimed that the Illuminati continued underground and were responsible for the French Revolution. It attracted literary men such as Johann Wolfgang von Goethe and Johann Gottfried'

In [23]:
answer

['The organizations were accused of conspiring to control world affairs and establish a New World Order. They were also accused of masterminding events and planting agents in governments and corporations to gain political power and influence.',
 'Adam Weishaupt was the non-clerical professor at the institution.',
 'The Bavarian Illuminati was founded by Adam Weishaupt on 1 May 1776.',
 'The Perfectibilists (early Illuminati) adopted the Owl of Minerva as their symbol.']

In [ ]:
## To dict
data={
    "question":questions,
    "ground_truth":ground_truth,
    "answer":answer,
    "contexts":content
}

In [25]:
from datasets import Dataset
dataset=Dataset.from_dict(data)

/home/shahul/miniconda3/envs/LCenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [26]:
dataset

Dataset({
    features: ['question', 'ground_truth', 'answer', 'contexts'],
    num_rows: 4
})

In [27]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

In [ ]:
result=evaluate(dataset=dataset,
                metrics=[
                    context_precision,
                    context_recall,
                    faithfulness,
                    answer_relevancy],
                llm=llm,
                embeddings=embeddings
                )


Evaluating: 100%|██████████| 16/16 [00:14<00:00,  1.12it/s]


In [29]:
result

{'context_precision': 1.0000, 'context_recall': 1.0000, 'faithfulness': 0.8750, 'answer_relevancy': 0.9247}

In [30]:
df=result.to_pandas()

In [31]:
df

,user_input,retrieved_contexts,response,reference,context_precision,context_recall,faithfulness,answer_relevancy
0,what were these organizations accused of?,"[In subsequent use, ""Illuminati"" has been used...",The organizations were accused of conspiring t...,The organizations were accused of conspiring t...,1.0,1.0,1.0,0.984643
1,who was the non-clerical professor at the inst...,[Adam Weishaupt (1748–1830) became professor o...,Adam Weishaupt was the non-clerical professor ...,Adam Weishaupt (1748–1830) became professor of...,1.0,1.0,1.0,0.792151
2,Who founded the Bavarian Illuminati and when?,"[With little difficulty, a warrant was obtaine...",The Bavarian Illuminati was founded by Adam We...,Adam Weishaupt founded the Bavarian Illuminati...,1.0,1.0,1.0,0.972597
3,What symbol did the Perfectibilists (early Ill...,"[Finding Freemasonry expensive, and not open t...",The Perfectibilists (early Illuminati) adopted...,The Perfectibilists adopted the Owl of Minerva...,1.0,1.0,0.5,0.949425
